In [241]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import sklearn as sk
from sklearn.model_selection import train_test_split
eps=np.finfo(float).eps
from binarytree import tree,Node
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score
from sklearn.tree import DecisionTreeClassifier
from operator import itemgetter
import copy
import collections
from pylab import *
import matplotlib
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [242]:
names=['ID','Age','Experience','Income(in p.a)','Zip','FamilySize','Expenditure(p.m)','EduLevel','MortageValue','Personalloan','SecurityAcc','CD account','InternetBanking','CreditCard']
df=pd.read_csv("LoanDataset/data.csv",names=names)
# df

,ID,Age,Experience,Income(in p.a),Zip,FamilySize,Expenditure(p.m),EduLevel,MortageValue,Personalloan,SecurityAcc,CD account,InternetBanking,CreditCard
0,2701,31,5,39,94590,4,2.20,2,0,0,0,0,1,1
1,2716,42,18,54,90089,1,1.80,1,0,0,0,0,1,0
2,3359,59,35,40,94536,4,0.40,1,0,0,0,0,0,0
3,2492,38,14,80,92868,2,2.70,1,0,0,0,0,1,0
4,937,62,32,19,92109,1,1.50,3,0,0,1,0,0,0
5,4546,61,35,35,90025,2,0.20,3,0,0,0,0,1,0
6,129,38,14,74,92182,2,0.00,1,0,0,1,0,1,0
7,1074,33,8,122,94583,1,0.00,1,0,0,0,0,1,1
8,4426,26,0,164,95973,2,4.00,3,301,1,0,0,1,0
9,1702,29,3,108,94304,4,1.80,2,0,0,0,0,0,0


In [243]:
numeric=['Age','Experience','Income(in p.a)','FamilySize','Expenditure(p.m)','EduLevel','MortageValue']
categoric=['Zip','Personalloan','SecurityAcc','CD account','InternetBanking','CreditCard']

In [244]:
X =df.drop(['Personalloan'],axis=1)
y=df['Personalloan']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
df=pd.concat([X_train,y_train],axis=1)

In [245]:
#Just seperation of class
def separateByClass(df):
#     print(dataset)
    separated = {}
    for index,row in df.iterrows():
        if  row['Personalloan'] in separated:
            separated[row['Personalloan']].append(list(row))
        else:
             separated[row['Personalloan']]=[]
    return separated

In [246]:
# sep=separateByClass(df)
# print(sep)

In [247]:
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [248]:
# sqrt((summa(x-avg)^2)/n)
def stdev(numbers):
    avg = mean(numbers)
    varianceNum=0.0
    for x in numbers:
        varianceNum=varianceNum+pow(x-avg,2)
    variance=varianceNum/float(len(numbers)-1)
#     variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [249]:
def summarize(dataset):
    summaries={}
    df1=pd.DataFrame(dataset)
    names=['ID','Age','Experience','Income(in p.a)','Zip','FamilySize','Expenditure(p.m)','EduLevel','MortageValue','SecurityAcc','CD account','InternetBanking','CreditCard','Personalloan']
    namesattr=['ID','Age','Experience','Income(in p.a)','Zip','FamilySize','Expenditure(p.m)','EduLevel','MortageValue','SecurityAcc','CD account','InternetBanking','CreditCard']
    df1.columns=names
#     print(df1)
    df1=df1.drop(['Personalloan'],axis=1)
    for attribute in namesattr:
        summaries[attribute]=(mean(df1[attribute]), stdev(df1[attribute]))
#         summaries.append((mean(df1[attribute]), stdev(df1[attribute])))
    return summaries

In [250]:
#har class ke har ek attribute ke liye mean and variance
def partitionByClass(df):
#     print(dataset)
    separated = separateByClass(df)
#     print("seperated")
#     print(seperated)
    summaries = {}
    #seperated by class list
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [251]:
#calculate gaussian distribution
def Gaussian(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

In [252]:
#calculate gaussian distribution of each attrbute in a given row
def getClassProbabilities(summaries,row):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for attr,meanvar in classSummaries.items():
            mean,stdev=meanvar
            x=row[attr]
            probabilities[classValue] *= Gaussian(x, mean, stdev)
    return probabilities

In [253]:
#most dominating attribute probablity
def getResultClass(summaries,row):
    probabilities = getClassProbabilities(summaries,row)
    if probabilities[0]>probabilities[1]:
        return 0
    else:
        return 1

In [254]:
def predict(summaries, testSet):
    predictions = []
    for index,row in testSet.iterrows():
        result = getResultClass(summaries,row)
        #result is basically in format of 1/0
        predictions.append(result)
    return predictions

In [255]:
# print(df)

In [256]:
summaries = partitionByClass(df)
# print(summaries)
# test model
# print(X_test)
predictions = predict(summaries, X_test)
# print(predictions)
# accuracy = getAccuracy(X_test, predictions)
# print('Accuracy: {0}%').format(accuracy)
print(confusion_matrix(y_test, predictions))  
print(classification_report(y_test,predictions)) 
print("Accuracy: ",accuracy_score(y_test, predictions))

[[700 103]
 [ 16  81]]
              precision    recall  f1-score   support

           0       0.98      0.87      0.92       803
           1       0.44      0.84      0.58        97

   micro avg       0.87      0.87      0.87       900
   macro avg       0.71      0.85      0.75       900
weighted avg       0.92      0.87      0.88       900

Accuracy: 0.8677777777777778


In [258]:
#Create a Gaussian Classifier
gnb = GaussianNB()

#Train the model using the training sets
gnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = gnb.predict(X_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 
# Model Accuracy, how often is the classifier correct?
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))

[[728  75]
 [ 41  56]]
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       803
           1       0.43      0.58      0.49        97

   micro avg       0.87      0.87      0.87       900
   macro avg       0.69      0.74      0.71       900
weighted avg       0.89      0.87      0.88       900

Accuracy: 0.8711111111111111
